In [1]:
# Import Data Profile Excel file name 'ProcessParasProfileUTL'
import pandas as pd
import os
from tqdm import tqdm

# Specify the path to your Excel file and the sheet names to import
#excel_path = 'your_excel_file.xlsx'
#sheet_names = ['Sheet1', 'Sheet2']  # Replace with your sheet names
excel_path = 'PPP.xlsx' # 'ProcessParasProfileUTL' sheet
sheet_names = ['ProcessParasProfileUTL']  # Replace with your sheet names
# Set the path to your folder containing Excel files
folder_path = 'myfolder'

# Read the specified sheets into a dictionary of DataFrames
dfs2 = pd.read_excel(
    excel_path,
    sheet_name=sheet_names,
    engine='openpyxl'  # Required for .xlsx files
)

# Access DataFrames using the sheet names as keys
# Example:
# df_sheet1 = dfs['Sheet1']
# df_sheet2 = dfs['Sheet2']
df4 = dfs2['ProcessParasProfileUTL']

# Initialize empty DataFrames to store data from all files
df3 = pd.DataFrame()

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):  # Process only .xlsx files
        excel_path = os.path.join(folder_path, filename)
        
        try:
            # Read both sheets from the Excel file
            dfs2 = pd.read_excel(
                excel_path,
                #sheet_name=['PPCDataUTL', 'ProcessParasProfileUTL'],
                engine='openpyxl'
            )
            
            # Append data to df3 and df4
            df3 = pd.concat([df3, dfs2], axis=0)
            
        except Exception as e:
            print(f"Error reading file {filename}: {e}")
            continue

# Optional: If you want to reset the index after concatenation
df3.reset_index(drop=True, inplace=True)
df3 = df3.drop(['EquipOpn','ULotID','EventID'], axis=1) #,'ActiveFlag'

# Clean 'ProcessParasProfileUTL' sheet on Duplicated column name
# Column index to use (e.g., column 0 = 'A')
column_index = 5 #5

# Extract values from the specified column (all rows)
column_values = df4.iloc[:, column_index].values  

# Create new DataFrame with these values as column names
new_df4 = pd.DataFrame(columns=column_values)

# Convert all columns in the new DataFrame to float32 (empty columns still need a data type)
new_df4 = new_df4.astype('float32')

# Clean ProcessParasProfileUTL sheet on Duplicated column name
new_df4 = new_df4.loc[:, ~new_df4.columns.duplicated()]

#combined_cols = pd.Index(df3.columns).append(new_df4.columns)
# Combine DataFrames vertically
combined_df = pd.concat([df3, new_df4], ignore_index=True)

# Initialize an empty dictionary
parameter = {}

# Convert the parameter values to float and prepare a row
row_data = {str(key): float(value) for key, value in parameter.items() if str(key) in combined_df.columns}

# Split the string into key-value pairs
def get_parameter(s):
    pairs = s.split(',')
    
    # Initialize an empty dictionary
    parameter = {}
    
    # Process each pair and convert to integers
    for pair in pairs:
        key_str, value_str = pair.split(':')
        key = int(key_str)
        if value_str != 'System.Byte[]':   # Skip key_str when value_str is undefined !!!!
            value = int(value_str)
            parameter[key] = value
            
    return(parameter)

# Update all rows in the DataFrame with values from the 'parameter' dictionary
for row_index in tqdm(combined_df.index, desc="Updating DataFrame rows"):
    parameter = get_parameter(df3.iloc[row_index, 2])
    for col in combined_df.columns:
        if col in parameter:
            combined_df.iloc[row_index, combined_df.columns.get_loc(col)] = parameter[col]

# Drop 'Parameter' column
combined_df = combined_df.drop(['Parameter'], axis=1)

# Save to csv file
combined_df.to_csv('output.csv', index=False)

Updating DataFrame rows: 100%|████████████████████████████████████████████████| 253220/253220 [21:31<00:00, 196.00it/s]


In [2]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253220 entries, 0 to 253219
Data columns (total 61 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   EquipID                 253220 non-null  object        
 1   Recipe                  253220 non-null  object        
 2   CreateTime              253220 non-null  datetime64[ns]
 3   EventDesc               253220 non-null  object        
 4   SAW_ProductionStock_Z1  248799 non-null  object        
 5   BladeOD_Z1              248799 non-null  float64       
 6   BladeThickness_Z1       248799 non-null  float64       
 7   FlangeODType_Z1         248799 non-null  object        
 8   SAW_ProductionStock_Z2  248799 non-null  object        
 9   BladeOD_Z2              248799 non-null  float64       
 10  BladeThickness_Z2       248799 non-null  float64       
 11  FlangeODType_Z2         248799 non-null  object        
 12  4280                    130248

In [6]:
combined_df.reset_index(drop=True, inplace=True)

In [7]:
new_df = combined_df[['EquipID','Recipe','CreateTime','EventDesc','4280','4290','1404','1405']]

KeyError: "['4280', '4290', '1404', '1405'] not in index"

In [4]:
combined_df

,EquipID,Recipe,CreateTime,EventDesc,SAW_ProductionStock_Z1,BladeOD_Z1,BladeThickness_Z1,FlangeODType_Z1,SAW_ProductionStock_Z2,BladeOD_Z2,...,1759,1755,1756,1500,1501,4204,1785,4205,1764,1766
0,TSWD184,SC90065P_A_2,2025-03-17 23:59:59.460,Kerf Check Start,SW00000057,2.1874,0.0010,Hub,SW00000003,2.1874,...,204.0,131.0,206.0,44960.0,44960.0,NaN,215.0,NaN,44960.0,44960.0
1,TSWD184,SC90065P_A_2,2025-03-17 23:59:59.413,Kerf Check Start,SW00000057,2.1874,0.0010,Hub,SW00000003,2.1874,...,NaN,NaN,NaN,NaN,NaN,45000.0,NaN,45000.0,NaN,NaN
2,TSWD176,EMS1204P_A,2025-03-17 23:59:58.900,Kerf Check Start,SW00000109,2.1874,0.0012,Hub,SW00000011,2.1874,...,211.0,137.0,205.0,45070.0,30010.0,NaN,210.0,NaN,45080.0,30010.0
3,TSWD176,EMS1204P_A,2025-03-17 23:59:58.890,Kerf Check Start,SW00000109,2.1874,0.0012,Hub,SW00000011,2.1874,...,NaN,NaN,NaN,NaN,NaN,45000.0,NaN,30000.0,NaN,NaN
4,TSWD163,ADI7712P_A_1,2025-03-17 23:59:58.763,Kerf Check End,SW00000057,2.1874,0.0010,Hub,SW00000067,2.1874,...,204.0,139.0,130.0,44960.0,29970.0,NaN,210.0,NaN,45000.0,29970.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253215,TSWD214,ONS2137P_E,2025-03-17 00:00:03.240,Kerf Check End,SW00000025,2.1874,0.0012,Hub,SW00000035,2.1874,...,NaN,NaN,NaN,NaN,NaN,45000.0,NaN,45000.0,NaN,NaN
253216,TSWD216,IDT4697P_A,2025-03-17 00:00:01.983,Kerf Check End,SW00000057,2.1874,0.0010,Hub,SW00000011,2.1874,...,206.0,133.0,206.0,49950.0,49950.0,NaN,209.0,NaN,49950.0,50000.0
253217,TSWD216,IDT4697P_A,2025-03-17 00:00:01.937,Kerf Check End,SW00000057,2.1874,0.0010,Hub,SW00000011,2.1874,...,NaN,NaN,NaN,NaN,NaN,50000.0,NaN,50000.0,NaN,NaN
253218,TSWD164,ONS2288P_A,2025-03-17 00:00:01.140,Kerf Check Start,SW00000057,2.1874,0.0010,Hub,SW00000130,2.1874,...,209.0,139.0,128.0,45000.0,44960.0,NaN,558.0,NaN,44960.0,44960.0
